<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/2_GraphSAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация GraphSAGE из PyTorch Geometric для классификации узлов на датасете Bitcoin-OTC с обработкой признаков рёбер

Задача — предсказать "уровень доверия" пользователей

Признаки узлов/рёбер - синтетические

Необходимо выбрать среду выполнения с GPU: Runtime → Change runtime type → GPU

## 1. Установка зависимостей

In [1]:
#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.2 MB/s eta 0:00:00


## 2. Импорт библиотек

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import BitcoinOTC
from torch_geometric.nn import SAGEConv
import numpy as np

## 3. Загрузка и подготовка данных

Обработка признаков:
* Нормализация степеней узлов
* Явное преобразование типов данных (float32 для рёбер)
* Использование среднего значения признаков рёбер для каждого узла

In [3]:
dataset = BitcoinOTC(root='data/BitcoinOTC', edge_window_size=10)
data = dataset[0]

Extracting data/BitcoinOTC/raw/soc-sign-bitcoinotc.csv.gz
Processing...
Done!


### 3.1 Создание синтетических признаков узлов (степень + нормализация)

In [4]:
degrees = np.zeros(data.num_nodes)
for edge in data.edge_index.t().tolist():
    degrees[edge[0]] += 1
data.x = torch.tensor(degrees, dtype=torch.float).view(-1, 1)
data.x = (data.x - data.x.mean()) / data.x.std()

### 3.2 Создание меток (3 класса) и обработка рёбер

In [5]:
labels = torch.zeros(data.num_nodes, dtype=torch.long)
q1 = np.quantile(degrees, 0.33)
q2 = np.quantile(degrees, 0.66)
labels[degrees > q2] = 2
labels[(degrees > q1) & (degrees <= q2)] = 1
data.y = labels
data.edge_attr = data.edge_attr.to(torch.float32)  # Явное преобразование типа

### 3.3 Разделение данных

In [6]:
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
indices = torch.randperm(data.num_nodes)
split = [0.6, 0.2, 0.2]
data.train_mask[indices[:int(split[0]*data.num_nodes)]] = True
data.val_mask[indices[int(split[0]*data.num_nodes):int((split[0]+split[1])*data.num_nodes)]] = True
data.test_mask[indices[int((split[0]+split[1])*data.num_nodes):]] = True

## 4. Определение модели GraphSAGE

Архитектура GraphSAGE:
* Два слоя SAGEConv с dropout-регуляризацией
* Объединение признаков узлов и усреднённых признаков рёбер
* Активация ReLU между слоями

In [7]:
class BitcoinSAGE(torch.nn.Module):
    def __init__(self, node_features, edge_hidden, hidden_channels, out_channels):
        super().__init__()
        # Кодировщик признаков рёбер
        self.edge_encoder = torch.nn.Linear(1, edge_hidden)  # 1 признак -> edge_hidden

        # SAGE слои
        self.conv1 = SAGEConv(node_features + edge_hidden, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.dropout = 0.5

    def forward(self, x, edge_index, edge_attr):
        # Агрегация признаков рёбер
        row = edge_index[0]
        edge_features = self.edge_encoder(edge_attr.view(-1, 1))  # [num_edges, edge_hidden]

        # Суммирование по узлам
        aggregated = torch.zeros(x.size(0), edge_features.size(1)).to(x.device)
        aggregated.scatter_add_(0, row.unsqueeze(-1).expand(-1, edge_features.size(1)), edge_features)

        # Объединение признаков
        x = torch.cat([x, aggregated], dim=1)  # [num_nodes, node_features + edge_hidden]

        # GraphSAGE
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout, training=self.training)
        return self.conv2(x, edge_index)

## 5. Инициализация модели

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BitcoinSAGE(
    node_features=data.x.size(1),  # 1
    edge_hidden=32,
    hidden_channels=64,
    out_channels=3
).to(device)

data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

## 6. Обучение модели

In [9]:
def accuracy(pred, true, mask):
    return (pred[mask].argmax(dim=1) == true[mask]).sum().float() / mask.sum()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Валидация
    model.eval()
    with torch.no_grad():
        val_acc = accuracy(out, data.y, data.val_mask)

    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d} | Loss: {loss.item():.4f} | Val Acc: {val_acc:.4f}')

Epoch 000 | Loss: 1.0535 | Val Acc: 0.8468
Epoch 020 | Loss: 0.0997 | Val Acc: 1.0000
Epoch 040 | Loss: 0.0018 | Val Acc: 1.0000
Epoch 060 | Loss: 0.0005 | Val Acc: 1.0000
Epoch 080 | Loss: 0.0005 | Val Acc: 1.0000
Epoch 100 | Loss: 0.0005 | Val Acc: 1.0000
Epoch 120 | Loss: 0.0005 | Val Acc: 1.0000
Epoch 140 | Loss: 0.0005 | Val Acc: 1.0000
Epoch 160 | Loss: 0.0005 | Val Acc: 1.0000
Epoch 180 | Loss: 0.0005 | Val Acc: 1.0000


## 7. Тестирование

In [10]:
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.edge_attr)
    test_acc = accuracy(out, data.y, data.test_mask)
    print(f'\nFinal Test Accuracy: {test_acc:.4f}')


Final Test Accuracy: 1.0000
